<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2024</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Subprocess, threading, multiprocessing</b><br/>
    <br/>
    <font>Никита Бондарцев</font><br/>
</center>

## Summary table

<table border="1" style="border-collapse: collapse; width: 100%;">
    <thead>
        <tr>
            <th>Сущность</th>
            <th>Доступ к памяти</th>
            <th>Контекст выполнения</th>
            <th>Сценарий использования</th>
            <th>Уровень изоляции</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Процесс</td>
            <td>Независимая память</td>
            <td>Управляется ОС</td>
            <td>Задачи с интенсивной загрузкой CPU, изолированные задачи</td>
            <td>Высокий (раздельная память)</td>
        </tr>
        <tr>
            <td>Поток</td>
            <td>Общая память внутри процесса</td>
            <td>Управляется ОС</td>
            <td>Конкурентные задачи с общими ресурсами</td>
            <td>Низкий (общая память)</td>
        </tr>
        <tr>
            <td>Корутина (следующая лекция)</td>
            <td>Общая память внутри потока</td>
            <td>Управляется программой (пользователем)</td>
            <td>Легковесная многозадачность (например, операции ввода-вывода)</td>
            <td>Низкий (общая память)</td>
        </tr>
    </tbody>
</table>


<center>
    <img src="images/multithreading_vs_multiprocessing.png" style="width: 800px"/>
    <div>
    <caption><a href="https://ycc.idv.tw/multithread-multiprocess-gil.html">https://ycc.idv.tw/multithread-multiprocess-gil.html</a> (careful, chinese)
    </caption></div>
</center>

# threading

**Поток** (тред; от англ. thread — нить) — поток выполнения команд внутри процесса

Потоки в рамках одного процесса делят **общую** память

<center>
<img src="http://www.openrtos.net/implementation/TaskExecution.gif" alt="concurrency" width="800px" />
</center>

В один момент времени **одно** ядро процессора исполняет **ровно один** поток

Несколько ядер могут выполнять несколько потоков буквально **одновременно**

<center>
<img src="https://www.backblaze.com/blog/wp-content/uploads/2017/08/diagram-thread-concurrency.png" alt="parallelism" width=800 />
</center>

Однако **не имеет смысла** создавать больше потоков, чем у вас есть ядер процессора (если целью является увеличение производительности)

## Let's finally try threading!

In [1]:
import threading

In [2]:
def greeter(num: int) -> None:
    print(f'Hello {num}')

In [3]:
def run_threads(count: int) -> None:
    threads = [
        threading.Thread(target=greeter, args=(i,))
        for i in range(count)
    ]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

In [4]:
run_threads(4)

Hello 0
Hello 1
Hello 2
Hello 3


Следующий пример взят из доклада Реймонда Хэттингера (Python core-developer):
https://www.youtube.com/watch?v=Bv25Dwe84g0

In [5]:
count = 0

def counter() -> None:
    global count
    count += 1
    print(f'{count} ', end='')

In [6]:
threads = [threading.Thread(target=counter) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

1 2 3 4 5 6 7 8 9 10 

В коде на слайде кроется ошибка, которая пока себя никак не проявляет

<center>
<img src="https://s3.amazonaws.com/s3-blogs.mentor.com/colinwalls/files/2018/05/RTC-520x118.png" alt="context-switching" />
</center>
Напоминание: потоки постоянно переключаются ОС

Давайте увеличим время жизни потока с помощью `time.sleep`

In [7]:
import time
import random

In [8]:
count = 0

def counter() -> None:
    global count
    old_count = count
    time.sleep(random.randint(0, 1))
    count = old_count + 1
    print(f'{count} ', end='')

In [9]:
%%time
count = 0
threads = [threading.Thread(target=counter) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

1 2 3 4 5 6 7 3 4 7 CPU times: user 4.97 ms, sys: 8.09 ms, total: 13.1 ms
Wall time: 1.01 s


Это извечная проблема многопоточного кода — **гонка** (от англ. **race**/**race condition**)

### Решение

In [10]:
count = 0
lock = threading.Lock()

def counter() -> None:
    global count
    with lock:
        old_count = count
        time.sleep(random.randint(0, 1))
        count = old_count + 1
        print(f'{count} ', end='')

In [11]:
%%time
threads = [threading.Thread(target=counter) for _ in range(10)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

1 2 3 4 5 6 7 8 9 10 CPU times: user 6.63 ms, sys: 8.8 ms, total: 15.4 ms
Wall time: 2.01 s


### Вариант с `queue.Queue`

<center>
<img src="https://pengphy.files.wordpress.com/2010/09/image5b95d.png?w=1400" alt="queue" width=500 />
</center>

### Задача
Посчитать сумму последовательности чисел в N потоков (<= кол-во ядер ЦПУ)

In [12]:
import queue
from collections.abc import Iterable

In [13]:
def adder(array: Iterable[int], part_id: int, thread_count: int, queue_out: queue.Queue) -> None:
    queue_out.put(sum(array[i] for i in range(part_id, len(array), thread_count)))

In [14]:
def sum_using_threads(array: Iterable[int], thread_count: int) -> list[int]:
    queue_out = queue.Queue()
    threads = [
        threading.Thread(target=lambda i=i: adder(array, i, thread_count, queue_out))
        for i in range(thread_count)
    ]
    for thread in threads:
        thread.start()
    results = []
    for thread in threads:
        results.append(queue_out.get())
        thread.join()
    return sum(results)

In [15]:
array = [1 for _ in range(10_000_000)]

In [16]:
%%timeit
sum(array[i] for i in range(len(array)))  # sum(array)

229 ms ± 578 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
sum_using_threads(array, 8)

218 ms ± 665 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Причина отсутствия ускорения: **GIL (Global Interpreter Lock)**

GIL делает питонячие потоки бесполезными для распараллеливания вычислений

### А когда есть смысл в питонячьих потоках?

In [18]:
import requests
urls = [
    'https://ya.ru', 'https://www.google.com',
    'https://www.python.org', 'https://isocpp.org',
    'https://habr.com', 'https://news.ycombinator.com'
]

In [19]:
def read_url(url: str) -> str:
    return requests.get(url).text

In [20]:
%%timeit
for url in urls:
    read_url(url)

3.53 s ± 361 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
readers = [
    threading.Thread(target=lambda url=url: read_url(url))
    for url in urls
]
for reader in readers:
    reader.start()
for reader in readers:
    reader.join()

893 ms ± 9.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Для специальных типов задач: **IO-bound**, питон-треды оказываются эффективны

<center>
<img alt="IO-bound" src="https://risingstack-blog.s3.amazonaws.com/2016/Jun/non_async_blocking_operations_example_in_node_hero_1459856858194-1466683867567.png" width=700 />
</center>

**NB**: Кроме того, иногда бывает полезным использовать треды для разделения потоков исполнения.

Хотя это и не даст выигрыша, организовывать сложный код таким образом может быть удобнее.

Например, отрисовка анимации, фоновые активности и прочие невысоконагруженные задачи.

### Random toy example to show logical separation of threads

In [22]:
import threading
import queue
import time
import random

random.seed(42)
# Shared queue for communication between threads
data_queue = queue.Queue()
# tricky question: can I use deque here instead? We do have the GIL after all?

def producer() -> None:
    """Simulates an indefinite data producer (e.g., reading from a sensor)"""
    for i in range(10):
        data = time.time()  # Simulated random data (timestamp)
        sleep = random.random() / 2 + 0.5
        print(f"Producer: Generated data {data}, sleeping for {sleep:5.3f}")
        data_queue.put(data)
        if sleep > 0.9:
            print("Sleep is larger than 0.9, exiting the producer")
            time.sleep(sleep)
            data_queue.put(None)
            break
        time.sleep(sleep)  # Simulate data delay, it releases the GIL

def consumer() -> None:
    """Simulates a data consumer (e.g., processing data)"""
    while True:
        data = data_queue.get()  # Get data from the queue, pauses if the queue is empty
        if data is None:  # Check for completion signal
            print("Consumer: Received stop signal. Exiting.")
            break
        print(f"Consumer: Processed data {data}")
        time.sleep(1.2)  # Simulate processing delay

# Create threads for producer and consumer
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

# Start threads
producer_thread.start()
consumer_thread.start()

# Wait for the threads to stop
producer_thread.join()  # we do not go further until the thread dies
print("Producer finished, message from the main thread")
consumer_thread.join()

print("Consumer stopped. Program finished.")


Producer: Generated data 1731977553.5941792, sleeping for 0.820
Consumer: Processed data 1731977553.5941792
Producer: Generated data 1731977554.4161122, sleeping for 0.513
Consumer: Processed data 1731977554.4161122
Producer: Generated data 1731977554.933107, sleeping for 0.638
Producer: Generated data 1731977555.5740821, sleeping for 0.612
Consumer: Processed data 1731977554.933107
Producer: Generated data 1731977556.1889532, sleeping for 0.868
Producer: Generated data 1731977557.062468, sleeping for 0.838
Consumer: Processed data 1731977555.5740821
Producer: Generated data 1731977557.904414, sleeping for 0.946
Sleep is larger than 0.9, exiting the producer
Consumer: Processed data 1731977556.1889532
Producer finished, message from the main thread
Consumer: Processed data 1731977557.062468
Consumer: Processed data 1731977557.904414
Consumer: Received stop signal. Exiting.
Consumer stopped. Program finished.


# subprocess

Иногда появляется необходимость **запустить некоторую программу из вашего питон-кода**

*Например*: наша тестирующая система запускает тесты для вашего кода. И нужно убедиться, что тесты отработали корректно

<center>
<img src="https://blag.felixhummel.de/_images/process_stdin_stdout_stderr_return-code.png" alt="process" width=800 />
</center>

In [23]:
import subprocess

In [ ]:
# Simplified code
print('Running tests...')
try:
    subprocess.run(
        ['pytest', '-p', 'no:cacheprovider', '--tb=no'],
        cwd=str(build_dir),
        timeout=60,
        check=True,
    )
except subprocess.CalledProcessError as exc:
    # Catch if any error occured
except subprocess.TimeoutExpired as exc:
    # or process timed out

In [ ]:
# recommended to use whenever it's possible
subprocess.run

# for more complex cases
subprocess.Popen

### Low-level approach (asynchroneous if needed)

In [ ]:
class Popen:
    def __init__(
        self,
        args,           # string or sequence to execute
        stdin=None, stdout=None, stderr=None,
        cwd=None,       # set current working directory
        env=None,       # set environment variables
        text=False,     # use default encoding
        shell=False,    # handle args as shell command
        ...             # and many other arguments
    ): ...

Документация: https://docs.python.org/3/library/subprocess.html

In [24]:
process = subprocess.Popen(['ls', '-l'])  # in practice, use os.listdir
print(process.poll())
# process

None
total 136
-rw-r--r--@ 1 nbond  staff  55082 Nov 19 04:52 SubprocessThreadingMultiprocessing.ipynb
drwxr-xr-x@ 4 nbond  staff    128 Nov 19 04:18 __pycache__
drwxr-xr-x  4 nbond  staff    128 Nov 19 02:38 images
-rw-r--r--@ 1 nbond  staff     49 Nov 19 04:18 multiply.py
-rw-r--r--@ 1 nbond  staff    672 Nov 16 15:57 rise.css
-rw-r--r--@ 1 nbond  staff     76 Nov 19 04:16 simple_append.py


In [25]:
# wait and get exit code
process.poll()

0

In [26]:
# wait with timeout
process.wait(timeout=1)  # timeout in secs

0

In [27]:
stdout, stderr = process.communicate()
stdout  # no stdout :(

### The PIPE to enable of communication


<center><img src="images/popen_pipe.webp"/><div><a href="https://florcvet.ru/python-subprocess-popen/">https://florcvet.ru/python-subprocess-popen/</a></div></center>

In [28]:
process = subprocess.Popen(['ls', '-l'], stdout=subprocess.PIPE, text=True)
stdout, _ = process.communicate()

In [29]:
print(''.join(stdout))

total 136
-rw-r--r--@ 1 nbond  staff  55091 Nov 19 04:54 SubprocessThreadingMultiprocessing.ipynb
drwxr-xr-x@ 4 nbond  staff    128 Nov 19 04:18 __pycache__
drwxr-xr-x  4 nbond  staff    128 Nov 19 02:38 images
-rw-r--r--@ 1 nbond  staff     49 Nov 19 04:18 multiply.py
-rw-r--r--@ 1 nbond  staff    672 Nov 16 15:57 rise.css
-rw-r--r--@ 1 nbond  staff     76 Nov 19 04:16 simple_append.py



In [ ]:
# communicate(input='...') with Popen(..., stdin=PIPE) to pass smth to stdin
# communicate(timeout=1) for setting timeout in seconds

In [ ]:
def run(
    *popenargs,
    check=False,           # check process exit code and raise on failure
    capture_output=False,  # set stdout=PIPE and stderr=PIPE
    timeout=None,          # passed to Popen.communicate(), raise on timeout
    input=None,            # passed to Popen.communicate() as stdin
    **popenkwargs
) -> subprocess.CompletedProcess: ...

In [30]:
subprocess.run(['bc'], input=b'2 * 3\n', capture_output=True)

CompletedProcess(args=['bc'], returncode=0, stdout=b'6\n', stderr=b'')

### subprocess - higher level, synchronous
This module intends to replace several older modules and functions:
```
os.system
os.spawn*
``` 
(c) python.org

see also: https://www.python.org/dev/peps/pep-0324/

На текущий момент `subprocess` - наиболее правильный способ запускать другие процессы на питоне

### Synchronous vs asynchronous

In [31]:
import subprocess
import time

# Define a shell command that takes time to complete
command = ["sleep", "5"]  # Sleeps for 5 seconds

print("Using subprocess.run (synchronous):")
start_time = time.time()

# Run the command synchronously
subprocess.run(command)  # Blocks until the sleep finishes
print(f"subprocess.run completed after {time.time() - start_time:.2f} seconds\n")

print("Using subprocess.Popen (asynchronous):")
start_time = time.time()

# Run the command asynchronously
process = subprocess.Popen(command)  # Starts the sleep and immediately returns
print(f"subprocess.Popen returned after {time.time() - start_time:.2f} seconds")

# Wait for the process to complete (optional)
process.wait()
print(f"subprocess.Popen process completed after {time.time() - start_time:.2f} seconds")


Using subprocess.run (synchronous):
subprocess.run completed after 5.03 seconds

Using subprocess.Popen (asynchronous):
subprocess.Popen returned after 0.01 seconds
subprocess.Popen process completed after 5.02 seconds


# multiprocessing

GIL запрещает параллельное исполнение нескольких потоков

<center>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/299675/images/1413349/Screen_Shot_2015-05-23_at_15.58.31.png" alt="GIL" width=600 />
</center>

Но GIL никак не может повлиять на разные процессы, поэтому в теории можно использовать разные процессы для распараллеливания вычислений!

In [32]:
# потому что не надо юзать макбук или винду ъуъ! На линуксе работает
# https://stackoverflow.com/questions/41385708/multiprocessing-example-giving-attributeerror
import multiprocessing
acc = []

def worker(accumulator) -> None:
    accumulator.append('item')

processes = [multiprocessing.Process(target=worker, args=(acc, )) for _ in range(5)]
for p in processes:
    p.start()
for p in processes:
    p.join()
    
accumulator

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/nbond/.pyenv/versions/3.12.5/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/nbond/.pyenv/versions/3.12.5/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/nbond/.pyenv/versions/3.12.5/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/nbond/.pyenv/versions/3.12.5/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/nbond/.pyenv/versions/3.12.5/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentin

NameError: name 'accumulator' is not defined

In [33]:
%%writefile simple_append.py

def worker(accumulator: list[str]) -> None:
    accumulator.append('item')


Overwriting simple_append.py


In [34]:
import multiprocessing
from simple_append import worker

acc = []

processes = [multiprocessing.Process(target=worker, args=(acc, )) for _ in range(5)]
for p in processes:
    p.start()
for p in processes:
    p.join()
    
acc

[]

Процессы не разделяют память, поэтому нужен механизм обмена данными между ними

Также **будьте аккуратны**! При передаче в подпроцесс чего-либо, происходит **pickle-unpickle**. Мало того, что это **не всегда возможно**, так еще и это может привести к **большому расходу памяти** и даже полной потере смысла от распараллеливания из-за копирования входных данных

Таким готовым механизмом обладает `multiprocessing.Pool`

In [35]:
%%writefile multiply.py

def multiplier(x: int) -> int:
    return x * 2

Overwriting multiply.py


In [36]:
from multiply import multiplier

with multiprocessing.Pool() as pool:
    result = pool.map(multiplier, range(10))
result

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Возвращаемся к задаче о суммировании

In [37]:
size = 10_000_000
array = [1 for _ in range(size)]

In [38]:
%%timeit
sum(array)

25.3 ms ± 287 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
process_count = multiprocessing.cpu_count()
print(f"You are a proud owner of {process_count} CPUs")
part_size = size // process_count
array_parts = [
    array[i * part_size: (i + 1) * part_size]
    for i in range(process_count)
]

You are a proud owner of 11 CPUs


In [41]:
with multiprocessing.Pool(process_count) as pool:
    %timeit pool.map(sum, array_parts)

92.1 ms ± 434 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
one_part = array[0 * part_size: (0 + 1) * part_size]
%timeit sum(one_part)

2.25 ms ± 3.94 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Много времени тратится на коммуникацию между процессами :(

Попробуем не пересылать много данных между процессами

In [45]:
%%writefile sum_module.py

def sum_n(n: int) -> int:
    return sum(1 for _ in range(n))

Overwriting sum_module.py


In [48]:
from sum_module import sum_n

In [49]:
%%timeit
sum_n(size)

208 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [50]:
from sum_module import sum_n
with multiprocessing.Pool(process_count) as pool:
    %timeit pool.map(sum_n, (part_size for _ in range(process_count)))

39.1 ms ± 688 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Есть заметный прирост в скорости, но множитель меньше, чем кол-во ядер ЦПУ :(

### Резюме

Создавать процессы — это дорого

Передавать данные между процессами — тоже дорого

Поэтому если данных для обмена много, и задача не слишком тяжелая, лучше обойтись без multiprocessing'а

В противном случае используйте multiprocessing — это просто

# И еще одна саммари табличка, тк легко запутаться

<table>
  <tr>
    <th>Функция</th>
    <th>threading.Thread</th>
    <th>multiprocessing.Process</th>
    <th>multiprocessing.Pool</th>
    <th>subprocess.run</th>
    <th>subprocess.Popen</th>
  </tr>
  <tr>
    <td>Контекст выполнения</td>
    <td>Общая память (в одном процессе)</td>
    <td>Раздельная память</td>
    <td>Раздельная память</td>
    <td>Внешний процесс</td>
    <td>Внешний процесс</td>
  </tr>
  <tr>
    <td>Параллелизм</td>
    <td>Квазипараллелизм (ограничен GIL)</td>
    <td>Истинный параллелизм</td>
    <td>Истинный параллелизм</td>
    <td>Истинный параллелизм</td>
    <td>Истинный параллелизм</td>
  </tr>
  <tr>
    <td>Лучшее применение</td>
    <td>I/O-задачи</td>
    <td>Задачи с высокой нагрузкой на CPU</td>
    <td>Высокоуровневый параллелизм задач</td>
    <td>Синхронное выполнение команд</td>
    <td>Асинхронное выполнение команд</td>
  </tr>
  <tr>
    <td>API</td>
    <td>.start(), .join()</td>
    <td>.start(), .join()</td>
    <td>.map(), .apply_async()</td>
    <td>.run()</td>
    <td>.Popen(), .communicate()</td>
  </tr>
  <tr>
    <td>Общая память</td>
    <td>Да</td>
    <td>Нет (используйте Manager/Queue)</td>
    <td>Нет (используйте Manager/Queue)</td>
    <td>Нет</td>
    <td>Нет</td>
  </tr>
  <tr>
    <td>Коммуникация</td>
    <td>Общие переменные</td>
    <td>Queue, Pipe, Manager</td>
    <td>Queue, Pipe, Manager</td>
    <td>Стандартные потоки</td>
    <td>Стандартные потоки</td>
  </tr>
  <tr>
    <td>Блокирующее выполнение</td>
    <td>Нет</td>
    <td>Нет</td>
    <td>Нет</td>
    <td>Да</td>
    <td>Нет</td>
  </tr>
  <tr>
    <td>Обработка ошибок</td>
    <td>Исключения Python</td>
    <td>Исключения Python</td>
    <td>Исключения Python</td>
    <td>Коды выхода</td>
    <td>Коды выхода, stderr</td>
  </tr>
  <tr>
    <td>Возврат результата</td>
    <td>Возвращает значения (вручную)</td>
    <td>Возвращает значения (вручную)</td>
    <td>Автоматически собирает результаты</td>
    <td>stdout (PIPE)</td>
    <td>stdout</td>
  </tr>
  <tr>
    <td>Завершение</td>
    <td>Нет terminate</td>
    <td>.terminate()</td>
    <td>Неприменимо</td>
    <td>Требуется ручное завершение</td>
    <td>.terminate()</td>
  </tr>
</table>


# Спасибо за внимание!